# Overview

# Origin

The Transformer model was born as a solution to address problems in the machine translation space. Machine translation, as the name suggests, is the process of using a machine (an ml model) to translate from one language into another.

## 2014 - (Cho et al) - Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation

- The RNN Encoder–Decoder neural network architecture was introduced
- 

## 2014 - (Sutskever et al) - Sequence to Sequence Learning with Neural Networks

- A multilayered Long Short-Term Memory (LSTM, a type of RNN)
- 

## 2017 - (Vaswani et al.) - Attention Is All You Need

In this paper the transformer model was proposed as an NN that outperforms RNNs

In the paper the transformer is used for machine translation (translating one language to another).

At the heart of this model is the novel concept of multi-head attention which is also introduced in the paper.

# Architecture

In the 2017 paper titled "Attention Is All You Need" The following architectural diagram is published:

<center><img src="./images/transformer_model_architecture.png" style="width:50%"></center>

# Implimentations